# Курсовая работа.
*Pandas и Matplotlib*


<br><br>
**Описание работы:**

Стоит задача подготовки аналитического отчета, который в дальнейшем поможет продюсерам образовательных программ эффективно выстраивать стратегию по модернизированию и улучшению курсов. В задачи входит оформление описательного блока по каждому курсу на основании рассчитанных показателей, расчет потенциальной нагрузки на преподавателей. Далее анализ качества контента курсов, выявление проблемных модулей. Также стоит задача выявить потенциальную сезонность. Каждый из пунктов анализа сопровождается аналитическим выводом на основании рассчитанных метрик.

<br><br>

_________

[Codebook](#Codebook) <br>
[1. Описание и начальная работа с данными](#1.1-Описание-и-начальная-работа-с-данными)<br>
[2. Расчет потенциальной нагрузки на преподавателей](#2.-Расчет-потенциальной-нагрузки-на-преподавателей)<br>
[3. Выявление проблемных модулей](#3.-Выявление-проблемных-модулей)<br>
[4. Расчет конверсии](#4.-Расчет-конверсии) <br>
[5. Метрика успеваемости ](#5.-Метрика-успеваемости)

## Codebook

`courses.csv` содержит следующие значения: <br><br>
&nbsp;&nbsp;&nbsp;&nbsp; `id` – идентификатор курса <br>
&nbsp;&nbsp;&nbsp;&nbsp; `title` – название курса <br>
&nbsp;&nbsp;&nbsp;&nbsp;  `field` – сфера, к которой относится курс <br> <br><br>
`students.csv` содержит следующие значения: <br><br>
&nbsp;&nbsp;&nbsp;&nbsp; `id` – идентификатор студента <br>
&nbsp;&nbsp;&nbsp;&nbsp; `city` – город студента <br>
&nbsp;&nbsp;&nbsp;&nbsp;  `birthday` – день рождения студента <br> <br><br>
`course_contents.csv` содержит следующие значения: <br><br>
&nbsp;&nbsp;&nbsp;&nbsp; `course_id` – идентификатор курса <br>
&nbsp;&nbsp;&nbsp;&nbsp; `module_number` – номер модуля <br>
&nbsp;&nbsp;&nbsp;&nbsp;  `module_title` – название модуля <br> 
&nbsp;&nbsp;&nbsp;&nbsp; `lesson_number` – номер урока <br>
&nbsp;&nbsp;&nbsp;&nbsp;  `lesson_title` – название урока <br>
&nbsp;&nbsp;&nbsp;&nbsp;  `lesson_token` – токен урока <br> 
&nbsp;&nbsp;&nbsp;&nbsp; `is_video` – наличие видео *(true/false)* <br>
&nbsp;&nbsp;&nbsp;&nbsp;  `is_homework` – наличие домашней работы *(true/false)* <br>
<br><br>
`progresses.csv` содержит следующие значения: <br><br>
&nbsp;&nbsp;&nbsp;&nbsp; `id` – идентификатор прогресса <br>
&nbsp;&nbsp;&nbsp;&nbsp; `student_id` – идентификатор студента <br>
&nbsp;&nbsp;&nbsp;&nbsp;  `course_id` – идентификатор курса <br> <br><br>
`progress_phases.csv` содержит следующие значения: <br><br>
&nbsp;&nbsp;&nbsp;&nbsp; `progress_id` – идентификатор прогресса <br>
&nbsp;&nbsp;&nbsp;&nbsp; `module_number` – номер модуля <br>
&nbsp;&nbsp;&nbsp;&nbsp; `lesson_number` – номер урока <br>
&nbsp;&nbsp;&nbsp;&nbsp;  `status` – статус прохождения урока <br>
&nbsp;&nbsp;&nbsp;&nbsp;  `start_date` – дата начала <br> 
&nbsp;&nbsp;&nbsp;&nbsp; `finish_date` – дата окончания <br>
<br><br>

## 1. Описание и начальная работа с данными

Нужно объединить датасеты (кроме `students.scv`) в один общий для дальнейшей работы. Отдельный датасет для расчета среднего возраста студентов курсов. <br> <br>
Необходимая информация: <br>
1. **(p)** Нужно посчитать:
      * общее количество курсов в датасете, 
      * количество модулей на каждом курсе, 
      * количество уроков в каждом модуле на каждом курсе, 
      * медианное количество уроков в модуле на каждом курсе, 
      * количество учеников на каждом курсе
      * минимальный, максимальный, средний, медианный возраст студентов
      * минимальный, максимальный, средний, медианный возраст студентов на каждом курсе
2. **(m)** Нужно построить bar-chart, отражающий количество студентов на каждом курсе.
3. **(m)** Нужно построить горизонтальный (столбцы должны располагаться горизонтально) bar-chart, отражающий количество студентов на каждом курсе. График должен иметь заголовок. Значения должны быть отсортированы. Цвет столбцов должен содержать информацию о сфере, к которой относится курс (то есть нужна легенда). Прозрачность должна стоять на отметке 0.1. На график должна быть нанесена линия медианы. У медианы должен быть свой цвет. Рамки у графика быть не должно ⭐
4.     На основании рассчитанных значений описать данные
_____________________________________________________________________


In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [3]:
# Открытие файлов 

df_main = pd.read_csv('datasets\progress_phases.csv')

df_progresses = pd.read_csv('datasets\progresses.csv')
df_progresses.rename(columns={'id': 'progress_id'}, inplace=True)

df_course_contents = pd.read_csv('datasets\course_contents.csv')
df_course_contents.drop(['course_id'], axis=1, inplace=True)

df_courses = pd.read_csv('datasets\courses.csv').iloc[:, 1:]
df_courses.rename(columns={'id':'course_id'}, inplace=True)

df_students = pd.read_csv('datasets\students.csv')


In [18]:
# Слияние таблиц в один датафрейм

df = pd.merge(df_main, df_progresses, on='progress_id', how='inner')
df = pd.merge(df, df_courses, on='course_id', how='inner')
df = pd.merge(df, df_course_contents, on=['module_number', 'lesson_number'], how='inner')
df

,progress_id,module_number,lesson_number,status,start_date,finish_date,student_id,course_id,title,field,module_title,lesson_title,lesson_token,is_video,is_homework
0,a387ab916f402cb3fbfffd29f68fd0ce,2,4,done,2018-06-23 08:28:50.681065+00,2018-06-23 08:28:52.439542+00,768c2987a744c51ce64a5993a2a94eaf,c9fcb746d51e41bc5a217002d081454b,Веб-дизайн PRO 2.0,Design,Синтаксис языка,Булева алгебра,f340f54e-437f-473b-8d93-b05a4fe990fa,True,False
1,a387ab916f402cb3fbfffd29f68fd0ce,2,4,done,2018-06-23 08:28:50.681065+00,2018-06-23 08:28:52.439542+00,768c2987a744c51ce64a5993a2a94eaf,c9fcb746d51e41bc5a217002d081454b,Веб-дизайн PRO 2.0,Design,Разработка стратегии продвижения в Social Media,Анализ конкурентов,39cf1c98-43b5-427b-b6b7-bc18340f46a6,True,False
2,a387ab916f402cb3fbfffd29f68fd0ce,2,4,done,2018-06-23 08:28:50.681065+00,2018-06-23 08:28:52.439542+00,768c2987a744c51ce64a5993a2a94eaf,c9fcb746d51e41bc5a217002d081454b,Веб-дизайн PRO 2.0,Design,Синтаксис языка,Циклы,2a1c95a0-2ad4-441a-a448-9da733a37f32,True,False
3,a387ab916f402cb3fbfffd29f68fd0ce,2,4,done,2018-06-23 08:28:50.681065+00,2018-06-23 08:28:52.439542+00,768c2987a744c51ce64a5993a2a94eaf,c9fcb746d51e41bc5a217002d081454b,Веб-дизайн PRO 2.0,Design,HTML,Таблицы и табличная вёрстка,b04758ba-e3ad-405e-aa53-e018218c74ec,True,False
4,a387ab916f402cb3fbfffd29f68fd0ce,2,4,done,2018-06-23 08:28:50.681065+00,2018-06-23 08:28:52.439542+00,768c2987a744c51ce64a5993a2a94eaf,c9fcb746d51e41bc5a217002d081454b,Веб-дизайн PRO 2.0,Design,Основной функционал Excel (продолжение),Инструмент «Текст по столбцам». Разделение дан...,51ba0ce8-6bb1-4d58-99fc-81143a9823b7,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3606749,176cb7e43a6314b0b897aead5a9b1bcf,5,13,done,2019-07-14 09:58:10.445895+00,2019-07-15 20:19:32.568717+00,c2e141029336bd476493cb68a10adb73,943306102e5b067d08a29094f37b8193,Java-разработчик c нуля,Development,Массивы и коллекции,Домашняя работа 5.5,c5564ee8-37c1-4f59-921e-a33b3e751a11,False,True
3606750,147c6db97dadc2aa92056bc14d6350df,5,13,done,2019-05-04 13:32:06.757255+00,2019-05-14 21:45:07.742472+00,e0034595fcbd27ac1c6d9aa027fd4f4c,943306102e5b067d08a29094f37b8193,Java-разработчик c нуля,Development,Массивы и коллекции,Домашняя работа 5.5,c5564ee8-37c1-4f59-921e-a33b3e751a11,False,True
3606751,1ad772d2e373085fd2836735a2261274,5,13,start,2019-07-15 08:57:51.739305+00,NaN,7244edba156d037b35bf477200cc8144,943306102e5b067d08a29094f37b8193,Java-разработчик c нуля,Development,Массивы и коллекции,Домашняя работа 5.5,c5564ee8-37c1-4f59-921e-a33b3e751a11,False,True
3606752,d35a065e6e023b0c7bb30b76f54f53ef,5,13,start,2019-07-10 20:56:19.980087+00,NaN,db75a51a0f39b2174ad5e913e2f83b4d,943306102e5b067d08a29094f37b8193,Java-разработчик c нуля,Development,Массивы и коллекции,Домашняя работа 5.5,c5564ee8-37c1-4f59-921e-a33b3e751a11,False,True


In [84]:
# Работа с датафреймом со студентами

df_st = df_students.dropna(axis=0)
df_stud = df[['student_id', 'course_id', 'title']].rename(columns={'student_id': 'id'})
df_st = df_st.merge(df_stud, on='id', how='inner')
df_st = df_st.drop_duplicates()
df_st['birthday'] = df_st['birthday'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
df_st

,id_,id,city,birthday,course_id,title
0,11070,35a36d90b745992b59085c8d54a072e7,Красноярск,1991-08-15 00:00:00,6e4837f0a65c68efbfdd0594d6774701,Java-разработчик
44,11756,37e8e8179295ae764524223086b0b568,Москва,1987-08-24 00:00:00,480d25ad36878fce09e7aada316ba1a6,Веб-разработчик
59,11756,37e8e8179295ae764524223086b0b568,Москва,1987-08-24 00:00:00,abce125a877c2196a3bc7bfbc11b5fc5,Веб-вёрстка для начинающих 2.0
74,11756,37e8e8179295ae764524223086b0b568,Москва,1987-08-24 00:00:00,17013cd19d25cb3f28dc1b2683721bb9,Веб-дизайн Базовый
580,13568,fb10649db552c5f451bea45ed754d428,Санкт-Петербург,1988-07-16 00:00:00,dad6f6ffc086caa89e2f40c28a9c7490,UX-дизайн
...,...,...,...,...,...,...
2670288,114136,0a5e2bb4947c56dfba3c60bd3af54a6f,Самара/Москва,1988-03-04 00:00:00,dad6f6ffc086caa89e2f40c28a9c7490,UX-дизайн
2670517,115361,d4b8116ba1d159c4b8b58ae77ad26578,Москва,1995-12-04 00:00:00,842218e7c983795934bbe4533c0888c8,SMM-маркетолог от А до Я
2670597,116725,3d7d520c6d739fc8cb844ebdc305743b,Москва,1990-02-21 00:00:00,dad6f6ffc086caa89e2f40c28a9c7490,UX-дизайн
2671111,119731,859b49d2953c191275061f41f1ba09be,Краснодар,1990-01-03 00:00:00,e9bb9205eeed307ee7cbaa08bfd166c3,JavaScript с нуля


In [98]:
# Определение среднего возраста для каждого курса

df_st['age'] = df_st['birthday'].apply(lambda x: (datetime.now() - x).days//365.25)
df_st.groupby('title').mean()['age']

title
Excel Базовый                          18.553719
Java-разработчик                       31.390543
Java-разработчик c нуля                31.035370
JavaScript с нуля                      32.957447
PHP-разработчик с 0 до PRO. Часть 1    34.092262
SMM-маркетолог от А до Я               31.577465
UX-дизайн                              32.010638
Анимация интерфейсов                   31.514583
Веб-вёрстка для начинающих 2.0         31.830609
Веб-дизайн PRO 2.0                     30.306802
Веб-дизайн Базовый                     30.611111
Веб-дизайн с нуля 2.0                  30.412417
Веб-разработчик                        30.167059
Интернет-маркетолог от Ingate          27.525876
Руководитель digital-проектов          33.545012
Name: age, dtype: float64

## 2. Расчет потенциальной нагрузки на преподавателей

1. **(p)** Рассчитать прирост студентов на каждом курсе в каждом месяце за всю историю (каждый месяц в диапазоне от марта 2016 до июля 2019 включительно). Считать дату начала прохождения курса студентом по дате начала первой домашней работы.
2. **(m)** На основании первого пункта построить line-graph с приростом студентов в каждом месяце для каждого курса. 15 графиков. Графики должны иметь заголовки, оси должны быть подписаны.
3. **(m)** На основании первого пункта построить line-graph с несколькими линиями, отражающими прирост студентов в каждом месяце для каждого курса. 15 линий на графике. График должен иметь заголовок. Ось, отражающая прирост, должна быть подписана. Линия для каждого курса должна иметь свой цвет (нужна легенда). Рамок у графика быть не должно ⭐
4. **(p)** Рассчитать количество прогрессов по выполнению домашних работ в каждом месяце за всю историю (каждый месяц в диапазоне от марта 2016 до июля 2019 включительно) для каждого курса. Учитывать, что выполнение домашнего задания может перетекать из одного месяца в другой (такие дз надо включать в общее число прогрессов для всех месяцев, которые покрывает срок выполнения этих дз)
5. **(m)** Построить line-graph по четвертому пункту. 15 графиков. Графики должны иметь заголовки, оси должны быть подписаны.
6. **(m)** Построить один line-graph для всех курсов по четвертому пункту. 15 линий на графике. График должен иметь заголовок. Ось, отражающая количество прогрессов, должна быть подписана. Линия для каждого курса должна иметь свой цвет (нужна легенда). Рамок у графика быть не должно ⭐
7. На основании рассчитанных значений сделать аналитический вывод (должен быть полным и покрывать все полученные выше метрики)

In [ ]:
### YOUR CODE HERE ###



## 3. Выявление проблемных модулей

1. **(p)** Рассчитать минимальное, максимальное, среднее, медианное время прохождения каждого модуля (разность между временем начала и окончания выполнения домашней работы) для каждого курса. Если домашних заданий в модуле несколько, то считать разность между временем начала выполнения первой домашней работы и временем окончания выполнения последней домашней работы в модуле
2. **(m)** На основании первого пункта построить line-graph с медианным временем прохождения каждого модуля для каждого курса. 15 графиков. Графики должны иметь заголовки
3. **(p)**  Чтобы выявить сезонность, посчитать медианное время выполнения домашней работы по месяцам (12 месяцев, январь-декабрь) для каждого курса. 
4. **(m)** На основании третьего пункта построить line-graph, на который будут нанесены линии для каждого курса с медианным временем выполнения домашней работы по месяцам. 15 линий на графике. График должен иметь заголовок. Ось, отражающая время прохождения, должна быть подписана. Линия для каждого курса должна иметь свой цвет (нужна легенда). Рамок у графика быть не должно  ⭐
5. На основании рассчитанных значений сделать аналитический вывод (должен быть полным и покрывать все полученные выше метрики)

In [ ]:
### YOUR CODE HERE ###




## 4. Расчет конверсии

1. **(p)** Посчитать конверсию перехода студентов из одного модуля в другой на каждом курсе. Формула: отношение количества студентов, приступивших к выполнению домашнего задания в этом модуле (если дз в модуле несколько, то считать по первому дз в модуле), к количеству студентов, сдавших задание в предыдущем модуле (если дз в модуле несколько, то считать по последнему дз в модуле).
2. **(m)** Построить bar-chart, отражающий конверсию перехода студентов из одного модуля в другой на каждом курсе. График должен иметь заголовок.
3. **(m)** Построить горизонтальный (столбцы должны располагаться горизонтально) bar-chart, отражающий конверсию перехода студентов из одного модуля в другой на каждом курсе. 15 графиков. Графики должны иметь заголовки. Ticks должны содержать номер и название модуля. Цвет столбцов графиков должен содержать информацию о сфере, к которой относится курс (нужна легенда). Прозрачность должна стоять на отметке 0.1. На графики должна быть нанесена линия медианы конверсии для каждого курса. У медианы должен быть свой цвет. Рамок у графиков быть не должно ⭐
4. На основании рассчитанных значений сделать аналитический вывод (должен быть полным и покрывать все полученные выше метрики)

In [ ]:
### YOUR CODE HERE ###




## 5. Метрика успеваемости 


Иногда студенты берут курсы, которые оказываются для них неподъемными. Это может быть как по причинам недостаточной изначальной подготовки, так и по причинам, связанным с низкой мотивацией студента. Для улучшения качества контента полезно выявить причину. На основании имеющихся данных придумать метрику успеваемости студента. **Обоснуйте ее.** Выявите таких студентов на каждом курсе, чтобы предоставить продюсерам список проблемных студентов. 

In [ ]:
### YOUR CODE HERE ###


